## Extracting from DB with SQL, Pandas

**To install the library:**   


In [56]:
! pip install -U pandasql  

# **test**

In [57]:
import pandas as pd
import numpy as np

In [58]:
df = pd.read_csv("./doc_use_log.csv")\
       .sample(frac=0.01, replace=False)

In [59]:
df.head()

,actiontype,ismydoc,ext,sessionid,documentposition,datetime
193350,OPEN,False,DOCX,486acd701c13cd4339617f32dc595e6b,OTHERAPP,2016.7.9
109887,CLOSE,True,XLS,d1c2817a5ff1bf4b0ad4c914850871c9,OTHERAPP,2016.7.21
100239,OPEN,True,HWP,31b9a5d8b42fd0b24b6177ec9f51c947,OTHERAPP,2016.7.11
221157,CLOSE,False,DOCX,0402e047bafe86a8f32160eca341fb51,OTHERAPP,2016.7.18
275949,CLOSE,True,DOC,01e78a53c271a3ec67122faadc0f08ff,OTHERAPP,2016.7.22


In [60]:
type(df)

pandas.core.frame.DataFrame

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3019 entries, 193350 to 76511
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   actiontype        3019 non-null   object
 1   ismydoc           3019 non-null   bool  
 2   ext               3019 non-null   object
 3   sessionid         3019 non-null   object
 4   documentposition  3019 non-null   object
 5   datetime          3019 non-null   object
dtypes: bool(1), object(5)
memory usage: 144.5+ KB


### About Pandasql
Reference: http://blog.yhat.com/posts/pandasql-sql-for-pandas-dataframes.html    

> locals() vs. globals()
pandasql needs to have access to other variables in your session/environment. You can pass locals() to pandasql when executing a SQL statement, but if you're running a lot of queries that might be a pain. To avoid passing locals all the time, you can add this helper function to your script to set globals() like so:

In [62]:
from pandasql import *

In [63]:
q = """
    SELECT * 
      FROM df 
     WHERE ext = 'PDF'
       AND ismydoc = '0'
     LIMIT 10
    """

In [64]:
print(sqldf(q, locals()).to_string())

  actiontype  ismydoc  ext                         sessionid documentposition   datetime
0       OPEN        0  PDF  1b5c388142297696f27fa2f3029df7e1         OTHERAPP  2016.7.23
1      CLOSE        0  PDF  415b278570d849927289b073d488edf3         OTHERAPP  2016.7.14
2      CLOSE        0  PDF  f9a2703d8429227287edc227f9098acb         OTHERAPP  2016.7.11
3       OPEN        0  PDF  7d03eb89d3a0128729f10134205f08c3     LOCALSTORAGE   2016.7.9
4       OPEN        0  PDF  15c2c517ef057c02decdf4b90dfde535       OTHERCLOUD  2016.7.13
5       OPEN        0  PDF  8976b75bcb2eb04254b4a0f2aeb94eda         OTHERAPP  2016.7.13
6      CLOSE        0  PDF  a4d9d10acf84579d6bf787b4598a1766         OTHERAPP   2016.7.4
7       OPEN        0  PDF  6bb660e67f03bf83d8426239d5ac0604         OTHERAPP  2016.7.17
8       OPEN        0  PDF  cebfd555bbf19901d18bf16071b4d21b         OTHERAPP  2016.7.20
9       OPEN        0  PDF  27745cb5a8b332671f9d19a43e2f6c8c         OTHERAPP  2016.7.21


In [65]:
# pandas
df[(df['ext'] == 'PDF') & (df['ismydoc'] == 0)].head(10)

,actiontype,ismydoc,ext,sessionid,documentposition,datetime
66144,OPEN,False,PDF,1b5c388142297696f27fa2f3029df7e1,OTHERAPP,2016.7.23
7914,CLOSE,False,PDF,415b278570d849927289b073d488edf3,OTHERAPP,2016.7.14
126824,CLOSE,False,PDF,f9a2703d8429227287edc227f9098acb,OTHERAPP,2016.7.11
9069,OPEN,False,PDF,7d03eb89d3a0128729f10134205f08c3,LOCALSTORAGE,2016.7.9
47815,OPEN,False,PDF,15c2c517ef057c02decdf4b90dfde535,OTHERCLOUD,2016.7.13
168151,OPEN,False,PDF,8976b75bcb2eb04254b4a0f2aeb94eda,OTHERAPP,2016.7.13
250706,CLOSE,False,PDF,a4d9d10acf84579d6bf787b4598a1766,OTHERAPP,2016.7.4
191227,OPEN,False,PDF,6bb660e67f03bf83d8426239d5ac0604,OTHERAPP,2016.7.17
12671,OPEN,False,PDF,cebfd555bbf19901d18bf16071b4d21b,OTHERAPP,2016.7.20
214099,OPEN,False,PDF,27745cb5a8b332671f9d19a43e2f6c8c,OTHERAPP,2016.7.21


In [66]:
# aggregation 집합
q = """
    SELECT ext,
           count(ext) as count,
           count(distinct sessionid) as unq_sess
      FROM df
     GROUP BY ext
     ORDER BY count DESC
     """

In [67]:
# sql문 해석

# df에서 unq_sess, count coulmn생성(distinct sessionid기준, *)
# ext를 pk로 하고
# count를 기준으로 내림차순하여 정렬

In [68]:
print(sqldf(q, locals()).to_string()) 

     ext  count  unq_sess
0    PDF    812       804
1   DOCX    559       558
2   XLSX    522       519
3    HWP    258       254
4    XLS    253       253
5    DOC    248       244
6   PPTX    174       172
7    TXT     98        98
8    PPT     62        60
9   PPSX     23        23
10   ODT      9         9
11   PPS      1         1


In [69]:
# pandas
df.groupby("ext").agg({"ext": "count", "sessionid": "nunique"})\
  .rename(columns={"ext": "ext_cnt", "sessionid": "session_cnt"}).sort_values("ext_cnt", ascending=False).reset_index()

,ext,ext_cnt,session_cnt
0,PDF,812,804
1,DOCX,559,558
2,XLSX,522,519
3,HWP,258,254
4,XLS,253,253
5,DOC,248,244
6,PPTX,174,172
7,TXT,98,98
8,PPT,62,60
9,PPSX,23,23


In [70]:
# Join
ios = pd.read_csv("./ios.csv")

ios.head()

,idx,sessionid,flag
0,1331,b8a35b63d65e36fcbcf7284db362fa06,iOS
1,1335,ab9ea758f63a61463efa059c7183b976,iOS
2,1383,401e95152f65bcd9573bb1707967e82a,iOS
3,1385,7891c7016850ccca28b54b3f8f0c554e,iOS
4,1387,dd16321402b19a3f20261e6905e4e9bf,iOS


In [71]:
ios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   idx        100 non-null    int64 
 1   sessionid  100 non-null    object
 2   flag       100 non-null    object
dtypes: int64(1), object(2)
memory usage: 2.5+ KB


In [72]:
len(ios.sessionid)

100

In [73]:
len(ios.sessionid.unique())

100

In [74]:
df.head()

,actiontype,ismydoc,ext,sessionid,documentposition,datetime
193350,OPEN,False,DOCX,486acd701c13cd4339617f32dc595e6b,OTHERAPP,2016.7.9
109887,CLOSE,True,XLS,d1c2817a5ff1bf4b0ad4c914850871c9,OTHERAPP,2016.7.21
100239,OPEN,True,HWP,31b9a5d8b42fd0b24b6177ec9f51c947,OTHERAPP,2016.7.11
221157,CLOSE,False,DOCX,0402e047bafe86a8f32160eca341fb51,OTHERAPP,2016.7.18
275949,CLOSE,True,DOC,01e78a53c271a3ec67122faadc0f08ff,OTHERAPP,2016.7.22


In [75]:
## select only ios log data
# duplicate check of ios table
q = """SELECT
            count(sessionid) as session_all_cnt,
            count(distinct sessionid) as session_uniq_cnt
        FROM ios
     """

In [76]:
print(sqldf(q, locals()))

   session_all_cnt  session_uniq_cnt
0              100               100


In [77]:
## join and select ios via flag col
q = """SELECT
            A.*, 
            B.flag
        FROM df A
            LEFT JOIN 
            (
            SELECT sessionid, flag
            FROM ios
            ) B
            ON A.sessionid = B.sessionid
        WHERE B.flag = 'iOS'    
     """

In [78]:
print(sqldf(q, locals()).to_string())

  actiontype  ismydoc   ext                         sessionid documentposition   datetime flag
0      CLOSE        0   PDF  401e95152f65bcd9573bb1707967e82a         OTHERAPP  2016.7.17  iOS
1       OPEN        1   DOC  04966493ec324c3a1e711b8967d5fec8         OTHERAPP  2016.7.14  iOS
2      CLOSE        0   PDF  0fe628b8406566d30791462be1e06300         OTHERAPP   2016.7.8  iOS
3      CLOSE        0  XLSX  15461f4fd396442f08872ef0c86688b3         OTHERAPP  2016.7.22  iOS
4       OPEN        1  XLSX  0ea96a02bceecbff38c0741474ebf703         OTHERAPP  2016.7.13  iOS


In [79]:
# with pandas
df.merge(ios, on="sessionid", how="left")\
  .query("flag =='iOS'")\
  .drop('idx', axis=1) #axis=0 열로 정리됨, axis=1 행으로 정리됨.

,actiontype,ismydoc,ext,sessionid,documentposition,datetime,flag
175,OPEN,True,XLSX,0ea96a02bceecbff38c0741474ebf703,OTHERAPP,2016.7.13,iOS
1053,OPEN,True,DOC,04966493ec324c3a1e711b8967d5fec8,OTHERAPP,2016.7.14,iOS
1760,CLOSE,False,XLSX,15461f4fd396442f08872ef0c86688b3,OTHERAPP,2016.7.22,iOS
2017,CLOSE,False,PDF,0fe628b8406566d30791462be1e06300,OTHERAPP,2016.7.8,iOS
2042,CLOSE,False,PDF,401e95152f65bcd9573bb1707967e82a,OTHERAPP,2016.7.17,iOS


### Question 1
- df 테이블의 Action Type 값(항목)별 유니크한 세션수는? 
- 유니크 세션수 기준으로 내림차순 정렬하기

In [80]:
df = pd.read_csv("./doc_use_log.csv").sample(frac=0.01, replace=False)

q = """
    SELECT actiontype,
           count(distinct sessionid) as unq_sess
      FROM df
     GROUP BY actiontype
     ORDER BY unq_sess DESC
     """

In [81]:
df.head()

,actiontype,ismydoc,ext,sessionid,documentposition,datetime
294635,OPEN,False,HWP,b18c52cfd52f8a85b729e086eda42fda,OTHERAPP,2016.7.20
288325,CLOSE,True,DOCX,bfcc17b556a274fd9ae0aeb9782a65d5,OTHERAPP,2016.7.12
190336,OPEN,True,DOCX,b0f4de689584a2f8f0cc006c0cda02b9,MYPOLARISDRIVE,2016.7.13
123897,OPEN,False,DOCX,1abe18b1ebc2bf75a4236c75bdd7cae3,OTHERAPP,2016.7.16
96649,CLOSE,True,PDF,91f442cb82e3245577d5db07bd868308,OTHERAPP,2016.7.25


In [82]:
print(sqldf(q, locals()))

  actiontype  unq_sess
0       OPEN      1495
1      CLOSE      1300
2       SAVE        97
3      RESET        79
4     SAVEAS        41


### **정답**

In [83]:
q = """
    SELECT actiontype,
           count (distinct sessionid) as session_cnt_uniq
      FROM df
     GROUP BY actiontype
     ORDER BY session_cnt_uniq DESC
     """
print(sqldf(q, locals()).to_string())

  actiontype  session_cnt_uniq
0       OPEN              1495
1      CLOSE              1300
2       SAVE                97
3      RESET                79
4     SAVEAS                41


In [106]:
# with pandas
df.query("ismydoc == 'false'").groupby("actiontype")['sessionid'].nunique().sort_values(ascending=False).reset_index()

,actiontype,sessionid


### Question 2   
- ismydoc이 1(True)인 경우에 한해, 날짜별 세션수의 유니크 빈도 구하기
- 유니크 빈도가 가장 큰 top 5 날짜 확인하기

In [107]:
q = """
    SELECT datetime,
           count (distinct sessionid) as session_cnt_uniq
      FROM df 
     WHERE ismydoc = '1'
     group by datetime
     order by session_cnt_uniq desc 
     LIMIT 5
    """
print(sqldf(q, locals()).to_string())

    datetime  session_cnt_uniq
0  2016.7.20                55
1  2016.7.27                50
2  2016.7.26                49
3  2016.7.22                48
4  2016.7.19                48


### **정답**

In [108]:
q = """
    SELECT datetime,
           count (distinct sessionid) as session_cnt_uniq
      FROM df
     WHERE ismydoc = '1'
     GROUP BY datetime
     ORDER BY session_cnt_uniq DESC
     LIMIT 5
    """
print(sqldf(q, locals()).to_string())

    datetime  session_cnt_uniq
0  2016.7.20                55
1  2016.7.27                50
2  2016.7.26                49
3  2016.7.22                48
4  2016.7.19                48


In [113]:
# with pandas
df.query("ismydoc == '1'").groupby("datetime")['sessionid'].nunique().sort_values(ascending=False).head(5).reset_index()

,datetime,sessionid


### Question 3
- 문서 포지션별(documentposition)로 자주 OPEN 되는 확장자(ext)를 확인하기
- 카운트 기준: unique sessionid
- 그룹별, 세션카운트 기준 desc 정렬

In [114]:
df.head()

,actiontype,ismydoc,ext,sessionid,documentposition,datetime
147040,RESET,True,PPT,7fb21bf73c9ffb361d408b723629b9f3,OTHERAPP,2016.7.17
51081,OPEN,True,PDF,9010fb4c329b3ade187f64a1631adc80,OTHERAPP,2016.7.22
287519,OPEN,True,PDF,7b9ce43b827869285d1affcccd252ff8,MYPOLARISDRIVE,2016.7.27
91085,CLOSE,True,HWP,ee73f250a15c88f2bc11f7efcde777f9,OTHERAPP,2016.7.7
124475,OPEN,False,PDF,22ffe6528b23326f8451268929075e9b,OTHERAPP,2016.7.1


In [119]:
q = """
    SELECT documentposition,
           ext,
           count(distinct sessionid) as session_cnt_uniq
      from df
    where actiontype = 'OPEN'
    group by documentposition, ext
    order by documentposition, session_cnt_uniq desc
    """
print(sqldf(q, locals()).to_string())

   documentposition   ext  session_cnt_uniq
0      LOCALSTORAGE  XLSX                18
1      LOCALSTORAGE   HWP                17
2      LOCALSTORAGE   PDF                12
3      LOCALSTORAGE  DOCX                12
4      LOCALSTORAGE   TXT                 9
5      LOCALSTORAGE   XLS                 6
6      LOCALSTORAGE  PPTX                 6
7      LOCALSTORAGE   DOC                 3
8    MYPOLARISDRIVE  XLSX                53
9    MYPOLARISDRIVE   PDF                51
10   MYPOLARISDRIVE  DOCX                47
11   MYPOLARISDRIVE   HWP                19
12   MYPOLARISDRIVE   XLS                17
13   MYPOLARISDRIVE   TXT                13
14   MYPOLARISDRIVE  PPSX                11
15   MYPOLARISDRIVE   DOC                11
16   MYPOLARISDRIVE  PPTX                 8
17   MYPOLARISDRIVE   PPT                 6
18   MYPOLARISDRIVE   ODT                 1
19      NEWDOCUMENT  DOCX                32
20      NEWDOCUMENT  XLSX                10
21      NEWDOCUMENT   HWP       

### **정답**

In [118]:
q = """
    SELECT documentposition,
           ext,
           COUNT (DISTINCT sessionid) as session_cnt_uniq
      FROM df
     WHERE actiontype = 'OPEN' 
     GROUP BY documentposition, ext
     ORDER BY documentposition, session_cnt_uniq DESC
     """
print(sqldf(q, locals()).to_string())

   documentposition   ext  session_cnt_uniq
0      LOCALSTORAGE  XLSX                18
1      LOCALSTORAGE   HWP                17
2      LOCALSTORAGE   PDF                12
3      LOCALSTORAGE  DOCX                12
4      LOCALSTORAGE   TXT                 9
5      LOCALSTORAGE   XLS                 6
6      LOCALSTORAGE  PPTX                 6
7      LOCALSTORAGE   DOC                 3
8    MYPOLARISDRIVE  XLSX                53
9    MYPOLARISDRIVE   PDF                51
10   MYPOLARISDRIVE  DOCX                47
11   MYPOLARISDRIVE   HWP                19
12   MYPOLARISDRIVE   XLS                17
13   MYPOLARISDRIVE   TXT                13
14   MYPOLARISDRIVE  PPSX                11
15   MYPOLARISDRIVE   DOC                11
16   MYPOLARISDRIVE  PPTX                 8
17   MYPOLARISDRIVE   PPT                 6
18   MYPOLARISDRIVE   ODT                 1
19      NEWDOCUMENT  DOCX                32
20      NEWDOCUMENT  XLSX                10
21      NEWDOCUMENT   HWP       

In [120]:
# with pandas
df.query("actiontype == 'OPEN'").groupby(['documentposition', 'ext'])['sessionid'].count().unstack().fillna(0)

ext,DOC,DOCX,HWP,ODT,PDF,PPS,PPSX,PPT,PPTX,TXT,XLS,XLSX
documentposition,,,,,,,,,,,,
LOCALSTORAGE,3.0,12.0,17.0,0.0,12.0,0.0,0.0,0.0,6.0,9.0,6.0,18.0
MYPOLARISDRIVE,11.0,47.0,19.0,1.0,51.0,0.0,11.0,6.0,8.0,13.0,17.0,53.0
NEWDOCUMENT,1.0,32.0,7.0,0.0,0.0,0.0,0.0,0.0,6.0,2.0,0.0,10.0
OTHERAPP,105.0,205.0,88.0,3.0,317.0,1.0,1.0,20.0,63.0,25.0,94.0,157.0
OTHERCLOUD,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
